In [8]:
from predictNew import LinkedinScraper

In [9]:
ethan = LinkedinScraper()

In [10]:
ethan.transform()

Please Enter Email  ethanpdoyle@gmail.com
Please Enter Password  ···········
Proceed? (yes / no) yes
Enter name:  ethan_profile
